In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib import font_manager, rc
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler

# 경고 메시지 무시
warnings.filterwarnings('ignore')

In [ ]:
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
from scipy.stats import kstest

In [ ]:
col_list = ['총자본증가율', '유형자산증가율', '유동자산증가율',
       '재고자산증가율', '자기자본증가율', '매출액증가율', '매출액영업이익률', '매출액순이익률', '자기자본순이익률',
       '자본금순이익률', '기업순이익률', '매출원가 대 매출액비율', '금융비용 대 총비용비율', '감가상각률',
       '차입금평균이자율', '사내유보율', '평균배당률', '자기자본배당률', '배당성향', '자기자본구성비율', '비유동비율',
       '비유동장기적합률', '유동비율', '당좌비율', '현금비율', '매출채권 대 매입채무비율', '부채비율', '유동부채비율',
       '비유동부채비율', '차입금의존도', '자기자본회전률', '자본금회전률', '매입채무회전률', '재고자산회전률',
       '비유동자산회전률', '유형자산회전율', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', '인건비(백만원)', '금융비용(백만원)', '임차료(백만원)', '조세공과(백만원)',
       '감가상각비(백만원)', '부채대비영업활동현금흐름', '자본대비영업활동현금흐름', '매출액대비영업활동현금흐름',
       '3개년가중평균매출액', '업력', 'label']

num_list = ['총자본증가율', '유형자산증가율', '유동자산증가율',
       '재고자산증가율', '자기자본증가율', '매출액증가율', '매출액영업이익률', '매출액순이익률', '자기자본순이익률',
       '자본금순이익률', '기업순이익률', '매출원가 대 매출액비율', '금융비용 대 총비용비율', '감가상각률',
       '차입금평균이자율', '사내유보율', '평균배당률', '자기자본배당률', '배당성향', '자기자본구성비율', '비유동비율',
       '비유동장기적합률', '유동비율', '당좌비율', '현금비율', '매출채권 대 매입채무비율', '부채비율', '유동부채비율',
       '비유동부채비율', '차입금의존도', '자기자본회전률', '자본금회전률', '매입채무회전률', '재고자산회전률',
       '비유동자산회전률', '유형자산회전율', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', '인건비(백만원)', '금융비용(백만원)', '임차료(백만원)', '조세공과(백만원)',
       '감가상각비(백만원)', '부채대비영업활동현금흐름', '자본대비영업활동현금흐름', '매출액대비영업활동현금흐름',
       '3개년가중평균매출액', '업력',]

In [ ]:
df_scaled = pd.read_csv('../datasets/비상장_train.csv', index_col = 0)[col_list]

### T-test(K-S통계량 검정)

In [ ]:
for j in num_list:
    a = kstest(df_scaled[j],'norm')
    p = a.pvalue
    print(j,a)

In [ ]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = df_scaled[df_scaled['label']== 1] #Existing Customer
Good = df_scaled[df_scaled['label']== 0] #Attrited Customer

c = []
for i in df_scaled.drop('label', axis = 1):
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    c.append([i,lresult[-1]])

c= pd.DataFrame(c)
c.columns=["피처값",'F-test']
c

In [ ]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
    else:
        c.loc[i,"분산"] = "hetero"
c

In [ ]:
c[c["분산"]=='homo']

In [ ]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        c.loc[i,"T-test"] = result[-1]
        print(Bad[c.loc[i,"피처값"]])
    else:
        c.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        c.loc[i,"T-test"] = result[-1]
c

In [ ]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
d = c[c["T-test"]<= 0.05]
print(d.sort_values('T-test',ascending=False)["피처값"].unique())
print('count :',d.sort_values('T-test',ascending=False)["피처값"].nunique())
# d.sort_values('T-test',ascending=False)["피처값"]

In [ ]:
t_test_feature = d.sort_values('T-test',ascending=True).dropna().iloc[:20,0].tolist()

### logit

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm
import numpy as np
lr_clf = LogisticRegression()

feature = df_scaled.drop('label',axis=1)
target = df_scaled['label']

logit = SelectFromModel(LogisticRegression())
logit.fit(feature, target)
logit_support = logit.get_support()
lr_feature = feature.loc[:,logit_support].columns.tolist()

In [ ]:
print(lr_feature)
print('count : ',len(lr_feature))

### Lasso

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [ ]:
# 12개 피처로 조정 C 값 조정
lasso = SelectFromModel(LogisticRegression(C=0.002, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lasso_feature = feature.loc[:,lasso_support].columns.tolist()

In [ ]:
print(lasso_feature)
print('count : ',len(lasso_feature))

### 후진제거법

In [ ]:
def backward_regression(X, y,
                           initial_list=[], 
                           threshold_out = 0.0000000000000001, # P-value 임계값 (제거 기준)
                           feature_list = num_list
                           ):
    
    
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adj_r_squared_list = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    included = feature_list
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] # 각 feature의 P값을 의미함
        worst_pval = pvalues.max()	# P 값이 가장 높은 것 선정
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
        
        step += 1
        steps.append(step)        
        adj_r_squared = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0).rsquared_adj
        adj_r_squared_list.append(adj_r_squared)
        sv_per_step.append(included.copy())
        
        if not changed:
            break
      
    return included,step,steps,adj_r_squared_list,sv_per_step

backward_valriables_function,step,steps,adj_r_squared_list,sv_per_step = backward_regression(feature, target)

In [ ]:
print(backward_valriables_function)
print('count :',len(backward_valriables_function))

### T-test / Lasso / Logit / 후진제거법을 통한 후보피쳐

In [ ]:
from collections import Counter

# 모든 리스트를 하나로 결합
combined_list = t_test_feature + lasso_feature + lr_feature + backward_valriables_function

# 항목의 등장 횟수 계산
item_count = Counter(combined_list)

# 두 번 이상 등장하는 항목 찾기
duplicates = [item for item, count in item_count.items() if count > 2]

print(duplicates)
print('count : ' ,len(duplicates))

### VIF, Corr, 도메인을 통한 최종피쳐 선택

In [ ]:
duplicates = ['매출액증가율', '자기자본증가율', 
              '매출액영업이익률', '매출액순이익률', '자기자본순이익률', '자본금순이익률', '기업순이익률', '금융비용 대 총비용비율', '사내유보율', 
              '자기자본구성비율', '당좌비율', '차입금의존도', 
              '인건비(백만원)', 
              '3개년가중평균매출액']

In [ ]:
def vif(data):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # VIF 출력을 위한 데이터 프레임 형성
    vif = pd.DataFrame()

    # VIF 값과 각 Feature 이름에 대해 설정
    vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif["features"] = data.columns 
                    
    # VIF 값이 높은 순으로 정렬
    vif = vif.sort_values(by="VIF Factor", ascending=False)
    vif = vif.reset_index().drop(columns='index')
    
    return vif

In [ ]:
vif(df_scaled[duplicates])

In [ ]:
# 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = df_scaled[duplicates]

# 상관관계 계산
correlation_matrix = df.corr()

# 상관 행렬의 위쪽 삼각형에 대한 마스크 생성
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# 히트맵 생성, 마스크 적용
plt.figure(figsize=(8, 6))  # 히트맵 크기 설정
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, mask=mask)
plt.title('비상장외감_후보피쳐_heatmap')  # 히트맵 제목 설정
plt.savefig('비상장외감_후보피쳐_heatmap.png')